In [82]:
import json
import base64
import gzip
import os
import shutil
import copy
from pathlib import Path
from typing import Union
from collections import defaultdict

In [83]:
DRAW_COMMANDS = [
    "drawArrays",
    "drawElements",
    "drawArraysInstanced",
    "drawArraysInstancedANGLE",
    "drawElementsInstanced",
    "drawElementsInstancedANGLE",
    "drawRangeElements",
    "multiDrawArraysWEBGL",
    "multiDrawElementsWEBGL",
    "multiDrawArraysInstancedWEBGL",
    "multiDrawElementsInstancedWEBGL",
    "multiDrawArraysInstancedBaseInstanceWEBGL",
    "multiDrawElementsInstancedBaseVertexBaseInstanceWEBGL",
]

STATES = [
    "BlendState",
    "ColorState",
    "CoverageState",
    "CullState",
    "DepthState",
    "DrawState",
    "PolygonOffsetState",
    "ScissorState",
    "StencilState",
    "VisualState",
    "DrawCall"
]

PURGE_KEYS = [
    'value',
    'displayText',
    # 'name'
]

In [84]:
def purge(co: Union[dict,list,int,str,float,bool,None]):
    if co is None:
        return None
    if isinstance(co, (int, str, float, bool)):
        return co
    if isinstance(co, list):
        return [purge(x) for x in co]
    if isinstance(co, dict):
        ret = {k: purge(v) for k, v in co.items() if k not in PURGE_KEYS and not k.startswith("__SPECTOR") and 'CommandId' not in k}
    return ret

In [85]:
obj = json.load(gzip.open('output/TEST-0_0_0.json.gz', 'rt'))

In [86]:
possible_pipelines = defaultdict(list)

In [87]:
for cmd in obj['commands']:
    if cmd['name'] in DRAW_COMMANDS:
        possible_pipelines[cmd['name']].append(purge({k: cmd[k] for k in STATES}))

In [88]:
type(possible_pipelines['drawElements'][0]['DrawCall']['uniforms'][0])

dict

In [89]:
possible_pipelines_set = defaultdict(set)

In [90]:
for k, v in possible_pipelines.items():
    for vv in v:
        # possible_pipelines_set[k].add(json.dumps(vv, sort_keys=True, separators=(",", ":"), indent=None))
        possible_pipelines_set[k].add(json.dumps(vv, sort_keys=True, indent=4))

In [91]:
print({k: len(v) for k, v in possible_pipelines.items()})
print({k: len(v) for k, v in possible_pipelines_set.items()})

{'drawElements': 730, 'drawArraysInstanced': 144}
{'drawElements': 38, 'drawArraysInstanced': 3}


In [92]:
shutil.rmtree('output/pipelines')
os.mkdir('output/pipelines')

for k, v in possible_pipelines_set.items():
    for i, vv in enumerate(v):
        with open(f'output/pipelines/{k}-{i}.json', 'wt') as f:
            f.write(vv)